## 1. 전처리

In [1]:
import re

def clean_text(text):
    """
    텍스트 데이터를 정리하는 함수:
    - 이모티콘 및 기타 기호 제거
    - 줄바꿈 문자 제거
    - HTML 태그 제거
    - URL 제거
    - 불필요한 공백 제거
    """
    # 이모티콘 및 특정 기호 패턴 (Unicode 범위를 사용)
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # 감정 표현 이모티콘
        "\U0001F300-\U0001F5FF"  # 기호 및 객체
        "\U0001F680-\U0001F6FF"  # 교통 및 기계
        "\U0001F700-\U0001F77F"  # 기타 기호
        "\U0001F780-\U0001F7FF"  # 추가적인 기호
        "\U0001F800-\U0001F8FF"  # 기호 확장
        "\U0001F900-\U0001F9FF"  # 이모티콘 확장
        "\U0001FA00-\U0001FA6F"  # 추가 기호
        "\U0001FA70-\U0001FAFF"  # 기타 범위
        "\u2600-\u26FF"          # 기타 기호
        "\u2700-\u27BF"          # 딩뱃
        "]",
        flags=re.UNICODE
    )

    # HTML 태그 제거
    text = re.sub(r'<[^>]+>', '', text)
    # URL 제거
    text = re.sub(r'http[s]?://\S+|www\.\S+', '', text)
    # 이모티콘 제거
    text = emoji_pattern.sub("", text)
    # 줄바꿈 문자 제거
    text = re.sub(r'\n', ' ', text)
    # 추가 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()

    return text


## 2. 파일 불러오기 및 전처리 적용

In [2]:
import pandas as pd
import json

with open('smartstore_reviews/smartstore_reviews/spiders/test1.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
df=pd.DataFrame(data)
df['review'] = df['review'].apply(clean_text)
df['product_name'] = df['product_name'].apply(clean_text)


## 3. 별점 개수 세기

In [11]:
# 별점 개수 세기
rating_counts = df['stars_score'].value_counts().sort_index()

print(rating_counts)


stars_score
1     13
2     10
3     25
4     84
5    338
Name: count, dtype: int64


In [10]:
# '배송' 키워드가 포함된 리뷰만 필터링
df_shipping = df[df['review'].str.contains('배송|포장', na=False)]

# 결과 확인
print(df_shipping.head())


    smartstore                                       product_name  product_id  \
2   vanessdeco  [베스트어워즈/무료배송] 바네스데코 대나무 햄퍼 다용도 원목 수납 1단 내추럴 빨래바구니  5058742474   
25  vanessdeco  [블프 세일가/무료배송] 바네스데코 대나무 햄퍼 다용도 원목 수납 1단 내추럴 빨래바구니  5058742474   
26  vanessdeco  [베스트어워즈/무료배송] 바네스데코 대나무 햄퍼 다용도 원목 수납 1단 내추럴 빨래바구니  5058742474   
43  vanessdeco  [블프 세일가/무료배송] 바네스데코 대나무 햄퍼 다용도 원목 수납 1단 내추럴 빨래바구니  5058742474   
50  vanessdeco  [블프 세일가/무료배송] 바네스데코 대나무 햄퍼 다용도 원목 수납 1단 내추럴 빨래바구니  5058742474   

                  product_option     user_id                     apply_date  \
2     상품: 01.1단 햄퍼(내추럴)(기본바스켓포함)    pan0****  2024-12-15T06:40:03.768+00:00   
25    상품: 01.1단 햄퍼(내추럴)(기본바스켓포함)    socc****  2024-11-26T14:33:07.448+00:00   
26  상품: 13.슬라이딩2칸선반(화이트)(이너박스포함)  dmsw******  2024-12-06T05:19:41.916+00:00   
43    상품: 01.1단 햄퍼(내추럴)(기본바스켓포함)     cds3***  2024-11-20T11:02:11.348+00:00   
50    상품: 01.1단 햄퍼(내추럴)(기본바스켓포함)   syun*****  2024-11-17T10:23:32.180+00:00   

                                      

## 4. 기존에 있던 db랑 합치기

In [12]:
final_df=pd.read_csv('final_df.csv')
df=pd.concat([df,rating_counts])
df

FileNotFoundError: [Errno 2] No such file or directory: 'final_df.csv'

In [75]:
# # 별점 개수 세기
# rating_counts = df['product_id'].value_counts().sort_index()
# df=df[df['product_id']=='352595429']
#
# print(rating_counts)


TypeError: '<' not supported between instances of 'int' and 'str'

## 5. 최소 리뷰 수 계산

In [77]:
# 별점-상품 조합의 최소 리뷰 수 계산
min_reviews_per_group = (
    df.groupby(['stars_score', 'product_id']).size().min()
)

# 별점-상품별로 균등 샘플링
sampled_df = (
    df.groupby(['stars_score', 'product_id'])
    .apply(lambda x: x.sample(9, random_state=42))
    .reset_index(drop=True)
)

# 별점별 리뷰 개수 확인
print(sampled_df.groupby('stars_score').size())

# 결과 출력
print(sampled_df)

stars_score
1    90
2    90
3    90
4    90
5    90
dtype: int64
         smartstore                  product_name  product_id  \
0           oastore            오아 무선 접이식 블루투스 키보드   352595429   
1           oastore            오아 무선 접이식 블루투스 키보드   352595429   
2           oastore            오아 무선 접이식 블루투스 키보드   352595429   
3           oastore  오아 무선 접이식 블루투스 키보드 OA-BTKBDA   352595429   
4           oastore  오아 무선 접이식 블루투스 키보드 OA-BTKBDA   352595429   
..              ...                           ...         ...   
445  homepowerkorea     퀸센스 스텐 인덕션 찜기 찜냄비 찜솥 28cm  6516046984   
446  homepowerkorea     퀸센스 스텐 인덕션 찜기 찜냄비 찜솥 32cm  6516046984   
447  homepowerkorea     퀸센스 스텐 인덕션 찜기 찜냄비 찜솥 32cm  6516046984   
448  homepowerkorea     홈파워 스텐 인덕션 찜기 찜냄비 찜솥 28cm  6516046984   
449  homepowerkorea      퀸센스 2단 스텐 찜냄비 찜솥 26cm 찜기  6516046984   

                              product_option       user_id  \
0                            옵션선택: 화이트 K0002     kimy*****   
1                           옵션

/var/folders/gb/6tdg7g8x441bp8yfgnfm4thh0000gn/T/ipykernel_64673/2661663775.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(9, random_state=42))


In [78]:
min_reviews_per_group

9

In [79]:
sampled_df

,smartstore,product_name,product_id,product_option,user_id,apply_date,review,stars_score,thumb_count
0,oastore,오아 무선 접이식 블루투스 키보드,352595429,옵션선택: 화이트 K0002,kimy*****,2022-07-12T23:57:24.726+00:00,전반적으로 좀 불편했습니다.,1,0
1,oastore,오아 무선 접이식 블루투스 키보드,352595429,옵션선택: 화이트 EV0737,shru********,2021-10-19T07:55:21.827+00:00,불편해서 잘 손에 안잡히네요...다른거 샀습니다...ㅎ,1,1
2,oastore,오아 무선 접이식 블루투스 키보드,352595429,옵션선택: 화이트 K0002,miya*****,2023-06-29T05:51:03.090+00:00,아직 몇개월 사용도 못해봤는데 벌써 고장이네요. 안돼요 ㅠㅠ,1,1
3,oastore,오아 무선 접이식 블루투스 키보드 OA-BTKBDA,352595429,옵션선택: 접이KB 화이트 K0002,sy_9*****,2019-11-28T08:33:35.000+00:00,일단 처음 사용하려고 열었는데 키가 하나 빠져있었고요.. 그건 잘 껴서 사용중인데 ...,1,1
4,oastore,오아 무선 접이식 블루투스 키보드 OA-BTKBDA,352595429,옵션선택: 접이KB 블랙 K0001,will****,2019-06-16T02:43:30.000+00:00,이렇게 배송온 물건 처음봤어요 포장하면서 분명 찢어진거 발견하셨을텐데 이정도는 대수...,1,8
...,...,...,...,...,...,...,...,...,...
445,homepowerkorea,퀸센스 스텐 인덕션 찜기 찜냄비 찜솥 28cm,6516046984,제품선택: 01-퀸센스 스텐 인덕션 찜기 찜냄비 찜솥 / 사이즈: 28,lin1***,2022-12-16T00:52:26.382+00:00,잘 받았습니다 아직 사용전,5,0
446,homepowerkorea,퀸센스 스텐 인덕션 찜기 찜냄비 찜솥 32cm,6516046984,제품선택: 01-퀸센스 스텐 인덕션 찜기 32cm / 사이즈: 32cm,urla******,2023-12-07T01:51:39.743+00:00,이걸로 손님오시면 숙주 배추 만두 두부넣고 오리고기 찜이나 대패삼겹이 찜으로 멋진 ...,5,0
447,homepowerkorea,퀸센스 스텐 인덕션 찜기 찜냄비 찜솥 32cm,6516046984,제품선택: 01-퀸센스 스텐 인덕션 찜기 28cm / 사이즈: 28cm,with*****,2024-01-20T06:45:31.426+00:00,찜기로 해서 집들이때 고기말이찜 잘 해서 먹었어요~ 사용하기 전에 잘 닦아서 사용해...,5,1
448,homepowerkorea,홈파워 스텐 인덕션 찜기 찜냄비 찜솥 28cm,6516046984,제품선택: 01-홈파워 스텐 인덕션 찜기 24cm / 사이즈: 24cm,bb****,2024-09-24T11:44:51.927+00:00,야채 쪄먹으려고 샀어요 잘 해먹어볼게요,5,0


In [80]:
sampled_df.to_csv('final_df.csv', index=False)

In [81]:
# product_id별 리뷰 개수 세기
rating_counts = sampled_df['product_id'].value_counts()

# 각 product_id와 개수 출력
for i, (product_id, count) in enumerate(rating_counts.items()):
    print(f'{i+1}번째, product_id: {product_id}, 개수: {count}')


1번째, product_id: 352595429, 개수: 45
2번째, product_id: 528580783, 개수: 45
3번째, product_id: 3063080859, 개수: 45
4번째, product_id: 4676444362, 개수: 45
5번째, product_id: 4711684205, 개수: 45
6번째, product_id: 5017491357, 개수: 45
7번째, product_id: 5155675165, 개수: 45
8번째, product_id: 5304277640, 개수: 45
9번째, product_id: 8901908546, 개수: 45
10번째, product_id: 6516046984, 개수: 45


In [70]:
# sampled_df=sampled_df[sampled_df['product_id']!=6455025636]